# Information about the transformer models used in the project:

This notebook gives the most basic information required to work with the transformer models used in the project, and some rationale for the choices of models to fine tune. For more information, refer to [The HuggingFace NLP Course](https://huggingface.co/learn/nlp-course/chapter0/1?fw=pt) 

## Token Classification:

The overacrching type of machine learning used in this project is called **"Named-Entity-Recogonition"** (**NER**). NER is used to extract words (or more generally, "tokens") from sentences/data. 

**Labeling scheme used:** A common labelling scheme used when doing NER uses 2\*n + 1  labels: O, B-entity1, I-entity1; B-entity2, I-entity2; ... B-entity_n, I-entity_n. **B** stands for **"beginning"** and is used to denote the beginning of an entity to extract; **I** stands **"inside"** and is used to label portions of an entity after the beginning token; and **O** stands for **other** and labels tokens that should not be extracted.

- example: Suppose you want to extract verbs and locations from sentences, then the labels would be B-verb, I-verb, O-verb; B-location, I-location, O-location. The sentence "I cashed my check at Bank of America." would be annotated "I *(O)* cashed *(B-verb)* my *(O)* check *(O)* at *(O)* Bank *(B-location)* of *(O)* America *(I-location)*."

In the current state of this project, n = 1, and whether or not an entity should be labeled is dictated by these [annotation guidelines](annotation_guidelines.md). In future iterations of the project, there will be a few different categories of entities, e.g. 'skills that the applicant should have', 'responsibilities that the person in the position has', etc. The model is stored on the HuggingFace Model Hub [here](https://huggingface.co/jfriduss/bert_for_job_descr_parsing), and is a fine tuned version of [this](https://huggingface.co/jjzha/jobbert_knowledge_extraction) model.

To use the model, the most basic elements to understand of it are (i) the way to convert the input to something the model can accept, and (ii) how to convert the outputs of the model to something useful. To see how this is done, consider the following test sentence below:

In [1]:
test_sentence = '4 + years of experience using finite element methods to understand fluids in extreme conditions'

**(i) Tokenizing the input:** First, the sample input is mapped to tokens that the model can process (this process is called "tokenization"). The next few cells show this being done:

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification

In [3]:
NER_checkpoint = "jfriduss/bert_for_job_descr_parsing"

token_class_mod = AutoModelForTokenClassification.from_pretrained(NER_checkpoint)
token_class_tok = AutoTokenizer.from_pretrained(NER_checkpoint)

In [4]:
tokenized_sentence = token_class_tok(test_sentence, padding = False, truncation = False, return_tensors="pt")
tokenized_sentence.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In theory, the model requires the tensors cooresponding to each of the keys in the above dictionary. But in this example, we only need the 'input_ids'. 'Token_type_ids' are only used in situations that require distinguishing between different parts of the input (e.g. if doing question-answering, distinguishing between the question and the answer) and the 'attention_mask' is used when processing multiple inputs of different sizes, at once, to tell the model that some input ids should be ignored. Because neither of these situations apply in this example--because NER in general doesn't typically require token type ids, and because, in the example, we are only dealing with one example sentence--only the input id's are used.

Because the model expects multiple sets of input ids, not just a single input, we need to create a tensor out of a list containing the tensor of input ids:

In [5]:
input_to_model = torch.tensor([tokenized_sentence['input_ids'][0].tolist()])
print("Below is an example of the input ids to input into the model:")
print(input_to_model)
print("\nNote for later that there are " + str(len(input_to_model[0])) + " input ids")

Below is an example of the input ids to input into the model:
tensor([[  101,   125,   116,  1201,  1104,  2541,  1606, 10996,  5290,  4069,
          1106,  2437, 24024,  1107,  6122,  2975,   102]])

Note for later that there are 17 input ids


**(ii) Understanding the model outputs:** 

Now, we feed the above into the model and look at the outputs:

In [6]:
model_output = token_class_mod(input_ids = input_to_model, )
print("Notice that the shape of the output from the model is " + str(model_output.logits[0].size()) + ".")
print("\nThis is intuitive: 17 in one dimension because 17 input ids, and 3 in the other dimension because "
     "there are three possible labels on each token ('B', 'I', or 'O').")

Notice that the shape of the output from the model is torch.Size([17, 3]).

This is intuitive: 17 in one dimension because 17 input ids, and 3 in the other dimension because there are three possible labels on each token ('B', 'I', or 'O').


To convert the rows of logits to probabilities of each label on each token, use softmax:

In [7]:
prediction = torch.nn.functional.softmax(model_output.logits, dim = -1)
prediction

tensor([[[9.1490e-03, 3.9810e-01, 5.9275e-01],
         [6.0002e-04, 5.2661e-03, 9.9413e-01],
         [1.2417e-03, 1.5245e-02, 9.8351e-01],
         [3.4219e-04, 4.4405e-03, 9.9522e-01],
         [1.3978e-04, 2.8973e-03, 9.9696e-01],
         [1.8221e-03, 1.2705e-02, 9.8547e-01],
         [4.8190e-04, 3.4519e-03, 9.9607e-01],
         [9.8048e-01, 1.6042e-02, 3.4811e-03],
         [8.7094e-04, 9.9877e-01, 3.5660e-04],
         [3.4989e-04, 9.9912e-01, 5.3096e-04],
         [3.8900e-04, 1.1448e-02, 9.8816e-01],
         [1.5848e-02, 7.2249e-02, 9.1190e-01],
         [4.3923e-01, 1.8263e-01, 3.7813e-01],
         [2.8913e-03, 7.9576e-01, 2.0135e-01],
         [1.1967e-01, 6.5570e-01, 2.2463e-01],
         [9.9405e-04, 9.6799e-01, 3.1019e-02],
         [9.1492e-03, 3.9809e-01, 5.9276e-01]]], grad_fn=<SoftmaxBackward0>)

And then, for each token/row, get the column with the max value to get the predicted label on that token/row.

In [8]:
torch.argmax(prediction, dim=-1)

tensor([[2, 2, 2, 2, 2, 2, 2, 0, 1, 1, 2, 2, 0, 1, 1, 1, 2]])

For information on aligning the predictions on the tokens, to predictions on words, see the function 'tok_sent_and_classify_toks' in the file [predict_and_label_helpers](../career_fit_tools/training/labeling_helpers/predict_and_label_helpers.py).

## Sentence Classification:

The project also uses a sentence classification model in front of the token classification model, to filter out sentences without any entities to extract. It uses the body of this [distilBERT model](https://huggingface.co/has-abi/distilBERT-finetuned-resumes-sections), that was fine tuned to classify sentences in resumes based on the section that the sentence occurs. A PyTorch linear layer was put on top of the transformer body, to convert the model into a binary classifier ('has entities to extract' or 'doesn't have entities to extract'). For information on the details of the model, refer to the [CustomModel](../career_fit_tools/training/ft_sentence_classification_helpers.py) class.

The basics of making a prediction using the model works similarly to using the token classification model, above:

First, load the model and tokenizer:

In [9]:
import os
import sys
from dotenv import load_dotenv

In [10]:
load_dotenv()
path_to_root_dir = os.getenv('PATH2ROOT_DIR')
sys.path.append(path_to_root_dir)

In [11]:
from career_fit_tools.training.ft_sentence_classification_helpers import CustomModel

In [12]:
path_to_sentence_classification_m = os.getenv('PATH2MODEL')

In [13]:
sentence_class_mod = CustomModel("has-abi/distilBERT-finetuned-resumes-sections", num_labels = 2)

p2model = path_to_sentence_classification_m + 'hf'
f_lin = path_to_sentence_classification_m + 'linear_layer_for_sent_classifier_fr_colab.pth'

#overwriting it with weights from the google colab
sentence_class_mod.overwrite_w_trained_weights(p2model, f_lin)
#uploading the sentence classification tokenizer
sentence_class_tok = AutoTokenizer.from_pretrained("has-abi/distilBERT-finetuned-resumes-sections")

Some weights of the model checkpoint at has-abi/distilBERT-finetuned-resumes-sections were not used when initializing DistilBertModel: ['classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


**(i) Tokenize the test sentence:**

In [14]:
tokd_sentence_sent_class = sentence_class_tok(test_sentence)
tokd_sentence_sent_class

{'input_ids': [11, 33, 24, 768, 231, 5852, 2269, 26060, 5356, 8749, 237, 16722, 20144, 230, 229, 14308, 4273, 12], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Create a list with each list above in it, and then convert each list to a tensor (this is done for the same reason as when did the same thing in token classification--because the model expects a batch of inputs):


In [15]:
tokd_sentence_sent_class['input_ids_tens'] = torch.Tensor([tokd_sentence_sent_class['input_ids']])
tokd_sentence_sent_class['attention_mask_tens'] = torch.Tensor([tokd_sentence_sent_class['attention_mask']])
tokd_sentence_sent_class['input_ids_tens']

tensor([[1.1000e+01, 3.3000e+01, 2.4000e+01, 7.6800e+02, 2.3100e+02, 5.8520e+03,
         2.2690e+03, 2.6060e+04, 5.3560e+03, 8.7490e+03, 2.3700e+02, 1.6722e+04,
         2.0144e+04, 2.3000e+02, 2.2900e+02, 1.4308e+04, 4.2730e+03, 1.2000e+01]])

Converts the data type of the tensor to int64, otherwise, an error is thrown when inputting the tensors into the model:

In [16]:
tokd_sentence_sent_class['input_ids_tens'] = tokd_sentence_sent_class['input_ids_tens'].to(torch.int64)
tokd_sentence_sent_class['attention_mask_tens'] = tokd_sentence_sent_class['attention_mask_tens'].to(torch.int64)

**(ii) Inputting the tensor into the model and understanding the models output:**

In [17]:
output_sent_class= sentence_class_mod.forward(input_ids=tokd_sentence_sent_class['input_ids_tens'],
                     attention_mask=tokd_sentence_sent_class['attention_mask_tens'])

Converting logits to probabilities:

In [18]:
prediction_sent_class = torch.nn.functional.softmax(output_sent_class.logits, dim=-1)
prediction_sent_class

tensor([[0.0112, 0.9888]], grad_fn=<SoftmaxBackward0>)

Getting the index with the max probability. Note that the model only outputs one row of probabilities, because it predicts a label on the entirety of the sentence, instead of on each token in the sentence, as occured above:

In [19]:
torch.argmax(prediction_sent_class, dim=1)

tensor([1])